In [1]:
!gdown https://drive.google.com/u/0/uc?id=1P2ixf9Re6Wr_XeZMikN-JW3E9UnyzULT&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=1P2ixf9Re6Wr_XeZMikN-JW3E9UnyzULT
To: /content/img_align_celeba.zip
100% 1.44G/1.44G [00:26<00:00, 53.9MB/s]


In [4]:
!unzip /content/img_align_celeba.zip -d /content/datasets/celeba

串流輸出內容已截斷至最後 5000 行。
 extracting: /content/datasets/celeba/img_align_celeba/197600.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197601.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197602.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197603.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197604.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197605.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197606.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197607.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197608.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197609.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197610.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197611.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197612.jpg  
 extracting: /content/datasets/celeba/img_align_celeba/197613.jpg  
 extracting: /content/datas

In [5]:
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import random
import os

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

# Root directory for dataset
dataroot = "datasets/celeba"
if not os.path.exists(dataroot):
    os.makedirs(dataroot)

image_size = 64
batch_size = 64
ngpu = 1

# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

Random Seed:  999


In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import models


# Decide which device we want to run on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Construct Model
z_dim = 128
netGen = models.Generator(z_dim).to(device)
netDis = models.Discriminator().to(device)
optGen = optim.Adam(netGen.parameters(), lr=4e-4, betas=(0.5, 0.999))
optDis = optim.Adam(netDis.parameters(), lr=1e-4, betas=(0.5, 0.999))

# Create Results Folder
model_name = "gan"
out_folder = "/content/drive/MyDrive/out/" + model_name + "/"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
save_folder =  "/content/drive/MyDrive/save/" + model_name + "/"
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Main Loop
num_epochs = 20
z_fixed = torch.randn(36, z_dim, device=device)
print("Start Training ...")
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        # Discriminator
        netGen.zero_grad()
        netDis.zero_grad()

        # TODO: Compute probability of real data and construct labels
        data = data[0].to(device)
        d_real = netDis(data)
        d_label = torch.ones_like(d_real).to(device)
        d_real_loss = nn.BCELoss()(d_real, d_label)

        # TODO: Compute probability of fake data and construct labels
        z = torch.randn(data.size(0), z_dim, device=device)
        d_fake = netDis(netGen(z).detach())
        d_label = torch.zeros_like(d_fake).to(device)

        d_fake_loss = nn.BCELoss()(d_fake, d_label)
        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        optDis.step()

        # Generator
        netGen.zero_grad()

        # TODO: Compute probability of fake data and construct labels
        d_fake = netDis(netGen(z))
        d_label = torch.ones_like(d_fake).to(device)

        g_loss = nn.BCELoss()(d_fake, d_label)
        g_loss.backward()
        optGen.step()

        # Show Information
        if i % 100 == 0:
            print("[%d/%d][%s/%d] D_loss: %.4f | G_loss: %.4f"\
            %(epoch+1, num_epochs, str(i).zfill(4), len(dataloader), d_loss.item(), g_loss.item()))

        if i % 500 == 0:
            print("Generate Images & Save Models ...")
            # Output Images
            x_fixed = netGen(z_fixed).cpu().detach()
            plt.figure(figsize=(6,6))
            plt.imshow(np.transpose(vutils.make_grid(x_fixed, nrow=6, padding=2, normalize=True).cpu(),(1,2,0)))
            plt.axis("off")
            plt.savefig(out_folder+str(epoch).zfill(2)+"_"+str(i).zfill(4)+".jpg", bbox_inches="tight")
            plt.close()
            # Save Model
            torch.save(netGen.state_dict(), save_folder+"netGen.pt")
            torch.save(netDis.state_dict(), save_folder+"netDis.pt")


Start Training ...
[1/20][0000/3166] D_loss: 1.3867 | G_loss: 0.7143
Generate Images & Save Models ...
[1/20][0100/3166] D_loss: 1.3754 | G_loss: 0.7487
[1/20][0200/3166] D_loss: 1.3728 | G_loss: 0.6921
[1/20][0300/3166] D_loss: 1.4145 | G_loss: 0.7401
[1/20][0400/3166] D_loss: 1.3434 | G_loss: 0.7205
[1/20][0500/3166] D_loss: 1.4107 | G_loss: 0.7182
Generate Images & Save Models ...
[1/20][0600/3166] D_loss: 1.3654 | G_loss: 0.7439
[1/20][0700/3166] D_loss: 1.3690 | G_loss: 0.7077
[1/20][0800/3166] D_loss: 1.3629 | G_loss: 0.8139
[1/20][0900/3166] D_loss: 1.4012 | G_loss: 0.8052
[1/20][1000/3166] D_loss: 1.3778 | G_loss: 0.6998
Generate Images & Save Models ...
[1/20][1100/3166] D_loss: 1.3611 | G_loss: 0.7628
[1/20][1200/3166] D_loss: 1.3651 | G_loss: 0.7363
[1/20][1300/3166] D_loss: 1.4258 | G_loss: 0.6924
[1/20][1400/3166] D_loss: 1.5317 | G_loss: 0.8898
[1/20][1500/3166] D_loss: 1.3381 | G_loss: 0.7256
Generate Images & Save Models ...
[1/20][1600/3166] D_loss: 1.3995 | G_loss: 0.

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import models

# Decide which device we want to run on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Construct Model
z_dim = 128
netEnc = models.Encoder(z_dim).to(device)
netDec = models.Generator(z_dim).to(device)
params = list(netEnc.parameters()) + list(netDec.parameters())
opt = optim.Adam(params, lr=2e-4, betas=(0.5, 0.999))

# Create Results Folder
model_name = "vae"
out_folder = "/content/drive/MyDrive/out/" + model_name + "/"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
save_folder =  "/content/drive/MyDrive/save/" + model_name + "/"
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Main Loop
num_epochs = 20
z_fixed = torch.randn(36, z_dim, device=device)
print("Start Training ...")
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        # Initialize
        netEnc.zero_grad()
        netDec.zero_grad()

        # Forward
        '''
        TODO: Forward Computation of the Network
        1. Compute z_mean and z_logvar.
        2. Sample z from z_mean and z_logvar.
        3. Compute reconstruction of x.
        '''
        data = data[0].to(device)
        z_mean, z_logvar = netEnc(data)
        z = torch.randn_like(z_mean).to(device) * torch.exp(0.5 * z_logvar) + z_mean
        recon_x = netDec(z)


        # Loss and Optimization
        '''
        TODO: Loss Computation
        rec_loss =
        kl_loss =
        '''
        rec_loss = F.binary_cross_entropy(recon_x, data)
        kl_loss = torch.mean(-0.5 * torch.sum(1 + z_logvar - z_mean**2 - z_logvar.exp(), 1))

        loss = rec_loss + 0.0001*kl_loss
        loss.backward()
        opt.step()

        # Show Information
        if i % 100 == 0:
            print("[%d/%d][%s/%d] R_loss: %.4f | KL_loss: %.4f"\
            %(epoch+1, num_epochs, str(i).zfill(4), len(dataloader), rec_loss.item(), kl_loss.mean().item()))

        if i % 500 == 0:
            print("Generate Images & Save Models ...")
            # Output Images
            x_fixed = netDec(z_fixed).cpu().detach()
            plt.figure(figsize=(6,6))
            plt.imshow(np.transpose(vutils.make_grid(x_fixed, nrow=6, padding=2, normalize=True).cpu(),(1,2,0)))
            plt.axis("off")
            plt.savefig(out_folder+str(epoch).zfill(2)+"_"+str(i).zfill(4)+".jpg", bbox_inches="tight")
            plt.close()
            # Save Model
            torch.save(netEnc.state_dict(), save_folder+"netEnc.pt")
            torch.save(netDec.state_dict(), save_folder+"netDec.pt")


Start Training ...
[1/20][0000/3166] R_loss: 0.7230 | KL_loss: 2.4620
Generate Images & Save Models ...
[1/20][0100/3166] R_loss: 0.5304 | KL_loss: 85.9830
[1/20][0200/3166] R_loss: 0.5308 | KL_loss: 102.5458
[1/20][0300/3166] R_loss: 0.5131 | KL_loss: 93.9969
[1/20][0400/3166] R_loss: 0.5242 | KL_loss: 93.8206
[1/20][0500/3166] R_loss: 0.5329 | KL_loss: 102.4714
Generate Images & Save Models ...
[1/20][0600/3166] R_loss: 0.5091 | KL_loss: 100.9299
[1/20][0700/3166] R_loss: 0.5281 | KL_loss: 96.6189
[1/20][0800/3166] R_loss: 0.5254 | KL_loss: 100.0506
[1/20][0900/3166] R_loss: 0.5067 | KL_loss: 100.7821
[1/20][1000/3166] R_loss: 0.5180 | KL_loss: 93.5986
Generate Images & Save Models ...
[1/20][1100/3166] R_loss: 0.5164 | KL_loss: 100.4677
[1/20][1200/3166] R_loss: 0.5013 | KL_loss: 95.0995
[1/20][1300/3166] R_loss: 0.5027 | KL_loss: 100.5746
[1/20][1400/3166] R_loss: 0.5098 | KL_loss: 93.9271
[1/20][1500/3166] R_loss: 0.5183 | KL_loss: 104.3666
Generate Images & Save Models ...
[1/20]